In [1]:
import spacy

In [2]:
import os
import argparse
import json
import sys
import torch
import numpy as np
import random

In [3]:
sys.path.insert(0, "Snippext_public")

In [ ]:
#os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [4]:
###karna adding 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")



Using device: cuda


In [5]:
from ditto_light.dataset import DittoDataset

/home/kbryan/miniconda3/envs/ditto38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from ditto_light.summarize import Summarizer

In [7]:
from ditto_light.knowledge import *

In [8]:
from ditto_light.ditto import train

In [9]:
import argparse
import sys

# Simulate command-line arguments in a notebook
sys.argv = [
    "notebook",
    "--task", "Structured/DBLP-ACM",
    "--batch_size", "32",
    "--max_len", "128",
    "--lr", "3e-5",
    "--n_epochs", "20",
    "--finetuning",
    "--lm", "roberta",
    "--fp16",
    "--da", "drop_col"
]

In [10]:
    parser = argparse.ArgumentParser()
    parser.add_argument("--task", type=str, default="Structured/Beer")
    parser.add_argument("--run_id", type=int, default=0)
    parser.add_argument("--batch_size", type=int, default=64)
    parser.add_argument("--max_len", type=int, default=256)
    parser.add_argument("--lr", type=float, default=3e-5)
    parser.add_argument("--n_epochs", type=int, default=20)
    parser.add_argument("--finetuning", dest="finetuning", action="store_true")
    parser.add_argument("--save_model", dest="save_model", action="store_true")
    parser.add_argument("--logdir", type=str, default="checkpoints/")
    parser.add_argument("--lm", type=str, default='distilbert')
    parser.add_argument("--fp16", dest="fp16", action="store_true")
    parser.add_argument("--da", type=str, default=None)
    parser.add_argument("--alpha_aug", type=float, default=0.8)
    parser.add_argument("--dk", type=str, default=None)
    parser.add_argument("--summarize", dest="summarize", action="store_true")
    parser.add_argument("--size", type=int, default=None)

_StoreAction(option_strings=['--size'], dest='size', nargs=None, const=None, default=None, type=<class 'int'>, choices=None, help=None, metavar=None)

In [11]:
hp = parser.parse_args()

In [12]:
    # set seeds
    seed = hp.run_id
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

In [13]:
torch.cuda.is_available()

True

In [14]:

    # only a single task for baseline
    task = hp.task

In [15]:

    # create the tag of the run
    run_tag = '%s_lm=%s_da=%s_dk=%s_su=%s_size=%s_id=%d' % (task, hp.lm, hp.da,
            hp.dk, hp.summarize, str(hp.size), hp.run_id)
    run_tag = run_tag.replace('/', '_')

In [16]:
    # load task configuration
    configs = json.load(open('configs.json'))
    configs = {conf['name'] : conf for conf in configs}
    config = configs[task]

    trainset = config['trainset']
    validset = config['validset']
    testset = config['testset']

In [ ]:
    # summarize the sequences up to the max sequence length
    if hp.summarize:
        summarizer = Summarizer(config, lm=hp.lm)
        trainset = summarizer.transform_file(trainset, max_len=hp.max_len)
        validset = summarizer.transform_file(validset, max_len=hp.max_len)
        testset = summarizer.transform_file(testset, max_len=hp.max_len)

In [ ]:
    if hp.dk is not None:
        if hp.dk == 'product':
            injector = ProductDKInjector(config, hp.dk)
        else:
            injector = GeneralDKInjector(config, hp.dk)

        trainset = injector.transform_file(trainset)
        validset = injector.transform_file(validset)
        testset = injector.transform_file(testset)

In [ ]:
    # load train/dev/test sets
    train_dataset = DittoDataset(trainset,
                                   lm=hp.lm,
                                   max_len=hp.max_len,
                                   size=hp.size,
                                   da=hp.da)
    valid_dataset = DittoDataset(validset, lm=hp.lm)
    test_dataset = DittoDataset(testset, lm=hp.lm)

/home/kbryan/miniconda3/envs/ditto38/lib/python3.8/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
    # train and evaluate the model
    train(train_dataset,
          valid_dataset,
          test_dataset,
          run_tag, hp)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/kbryan/miniconda3/envs/ditto38/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/kbryan/miniconda3/envs/ditto38/lib/python3.8/site-packages/apex/__init__.py:68: DeprecatedFeatureWarning: apex.amp is deprecated and will be removed by the end of February 2023. Use [PyTorch AMP](https://pytorch.org/docs/stable/amp.html)
  warnings.warn(msg, DeprecatedFeatureWarning)
/home/kbryan/miniconda3/envs/ditto38/lib/python3.8/site-packages/apex/amp/scaler.py:

Selected optimization level O2:  FP16 training with FP32 batchnorm and FP32 master weights.

Defaults for this optimization level are:
enabled                : True
opt_level              : O2
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : True
master_weights         : True
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O2
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : True
master_weights         : True
loss_scale             : dynamic
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
step: 0, loss: 0.6026742458343506
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0


/home/kbryan/miniconda3/envs/ditto38/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


step: 10, loss: 0.5332529544830322
step: 20, loss: 0.5093533992767334
step: 30, loss: 0.29905033111572266
step: 40, loss: 0.11101318895816803
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
step: 50, loss: 0.26841580867767334
step: 60, loss: 0.22708538174629211
step: 70, loss: 0.2664943337440491
step: 80, loss: 0.10532033443450928
step: 90, loss: 0.05832894146442413
step: 100, loss: 0.052763551473617554
step: 110, loss: 0.04017935320734978
step: 120, loss: 0.1533481776714325
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0
step: 130, loss: 0.10206272453069687
step: 140, loss: 0.14879921078681946
step: 150, loss: 0.004281593486666679
step: 160, loss: 0.0412931889295578
step: 170, loss: 0.02741059474647045
step: 180, loss: 0.023032033815979958
step: 190, loss: 0.0023414466995745897
step: 200, loss: 0.010561375878751278
step: 210, loss: 0.0042907665483653545
step: 220, loss: 0.0015992473345249891
step: 230, loss: 0.0150621188804

/home/kbryan/miniconda3/envs/ditto38/lib/python3.8/site-packages/apex/amp/_initialize.py:27: UserWarning: An input tensor was not cuda.
  warnings.warn("An input tensor was not cuda.")


...

epoch 19: dev_f1=0.988814317673378, f1=0.9887640449438202, best_f1=0.9821826280623607


/home/kbryan/miniconda3/envs/ditto38/lib/python3.8/site-packages/apex/amp/_initialize.py:27: UserWarning: An input tensor was not cuda.
  warnings.warn("An input tensor was not cuda.")


step: 0, loss: 8.09863195172511e-06
step: 10, loss: 9.081430471269414e-05
step: 20, loss: 2.946704080386553e-05
step: 30, loss: 3.865132021019235e-05
step: 40, loss: 4.7980349336285144e-05
step: 50, loss: 0.0022739218547940254
step: 60, loss: 7.243680011015385e-05
step: 70, loss: 0.00018401589477434754
step: 80, loss: 7.043209916446358e-05
step: 90, loss: 7.714896128163673e-06
step: 100, loss: 3.751469193957746e-05
step: 110, loss: 2.5653833290562034e-05
step: 120, loss: 0.00014187698252499104
step: 130, loss: 4.810591053683311e-05
step: 140, loss: 0.00011062448174925521
step: 150, loss: 0.0071466900408267975
step: 160, loss: 2.6854359020944685e-05
step: 170, loss: 0.025388887152075768
step: 180, loss: 0.001824264181777835
step: 190, loss: 6.440075958380476e-05
step: 200, loss: 1.651348611630965e-05
step: 210, loss: 2.14629435504321e-05
step: 220, loss: 0.0003076482389587909
step: 230, loss: 3.110334000666626e-05


/home/kbryan/miniconda3/envs/ditto38/lib/python3.8/site-packages/apex/amp/_initialize.py:27: UserWarning: An input tensor was not cuda.
  warnings.warn("An input tensor was not cuda.")


epoch 20: dev_f1=0.988814317673378, f1=0.9887640449438202, best_f1=0.9821826280623607
